In [1]:
import os
GCP_PROJECT_ID=os.getenv('PROJECT_ID')
GCS_BUCKET_NAME=os.getenv('BUCKET_NAME')

In [2]:
import sys
sys.path.append(os.path.join(os.getcwd(), '../'))


In [3]:
# read the dataset 
from cloud.gcs_storage_operations import GCSDataOperations
gcs = GCSDataOperations(GCP_PROJECT_ID)
df = gcs.read_parquet(GCS_BUCKET_NAME, "clean_data.parquet")

INFO:cloud.gcs_storage_operations:GCS Data client initialized successfully
INFO:cloud.gcs_storage_operations:Parquet read successfully (296858 rows, 15 columns)


In [4]:
# df.drop(columns=['description','id'], inplace=True)

In [5]:
# df = df[df['price']<=70000]

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
# feature_columns=['year', 'manufacturer', 'model', 'fuel', 'odometer',
#        'title_status', 'transmission', 'drive', 'type', 'paint_color', 'state',
#        'lat', 'long', 'census_region']

# target_column='price'

In [8]:
# # Prepare features and target
# X = data[feature_columns]
# y = data[target_column]

In [9]:
# print(f"Dataset shape: {X.shape}")
# print(f"Target variable: {target_column}")
# print(f"Features: {len(feature_columns)} total")

In [10]:
# lets write feature engineering scripts  

# version 1 : raw dataset without any feature engineering 
df_version_1 = df.copy()


In [11]:
# version 2 : dataset with encoding  one hot encoding and standard scaling  

df_version_2 = df.copy()

df_version_2.head()

,price,year,manufacturer,model,fuel,odometer,title_status,transmission,drive,type,paint_color,state,lat,long,census_region
27,33590,2014.0,gmc,Sierra,gas,57923.0,clean,automatic,4wd,pickup,white,al,32.590,-85.4800,East South Central
28,22590,2010.0,chevrolet,silverado,gas,71229.0,clean,automatic,4wd,pickup,blue,al,32.590,-85.4800,East South Central
29,39590,2020.0,chevrolet,silverado,gas,19160.0,clean,automatic,4wd,pickup,red,al,32.590,-85.4800,East South Central
30,30990,2017.0,toyota,Tundra,gas,41124.0,clean,automatic,4wd,pickup,red,al,32.590,-85.4800,East South Central
31,15000,2013.0,ford,F-150,gas,128000.0,clean,automatic,rwd,truck,black,al,32.592,-85.5189,East South Central


In [12]:
# Encoing  data
from sklearn.preprocessing import LabelEncoder

categorical_columns = ['manufacturer', 'model', 'fuel', 'title_status', 
                          'transmission', 'drive', 'type', 'paint_color', 'state', 'census_region']
    
label_encoders = {}
encoding_metadata = {"encoders": {}}

for col in categorical_columns:
        if col in df_version_2.columns:
            print(f"\n📊 Processing column: '{col}'")
            
            # Create and fit label encoder
            le = LabelEncoder()
            df_version_2[col] = le.fit_transform(df_version_2[col])
            label_encoders[col] = le
            
            # Create detailed mapping metadata
            value_to_encoded = dict(zip(le.classes_, le.transform(le.classes_)))
            encoded_to_value = {v: k for k, v in value_to_encoded.items()}
            
            # Get statistics
            unique_count = len(le.classes_)
            most_frequent = df_version_2[col].mode().iloc[0] if not df_version_2[col].empty else None
            most_frequent_original = encoded_to_value.get(most_frequent, "Unknown")
            
            # Store metadata for this column
            column_metadata = {
                "original_values": le.classes_.tolist(),
                "unique_count": unique_count,
                "value_to_encoded_mapping": value_to_encoded,
                "encoded_to_value_mapping": encoded_to_value,
                "encoded_range": f"0 to {unique_count - 1}",
                "most_frequent_encoded": int(most_frequent) if most_frequent is not None else None,
                "most_frequent_original": most_frequent_original,
                "data_type": "categorical → numerical (label_encoded)"
            }
            
            encoding_metadata["encoders"][col] = column_metadata
            


📊 Processing column: 'manufacturer'

📊 Processing column: 'model'

📊 Processing column: 'fuel'

📊 Processing column: 'title_status'

📊 Processing column: 'transmission'

📊 Processing column: 'drive'

📊 Processing column: 'type'

📊 Processing column: 'paint_color'

📊 Processing column: 'state'

📊 Processing column: 'census_region'


In [17]:
import json
import numpy as np

def json_serializer(obj):
    """Convert numpy types to native Python types for JSON serialization"""
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif hasattr(obj, 'tolist'):
        return obj.tolist()
    return str(obj)

def convert_keys_to_strings(obj):
    """Recursively convert dictionary keys to strings"""
    if isinstance(obj, dict):
        return {str(k): convert_keys_to_strings(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_keys_to_strings(item) for item in obj]
    else:
        return obj

# Convert the metadata before saving
encoding_metadata_clean = convert_keys_to_strings(encoding_metadata)

# save to gcp : 
gcs.upload_json(bucket_name=GCS_BUCKET_NAME,blob_name='metadata/encoding/df_version_2_label_encoding_metadata.json',data=encoding_metadata_clean)

INFO:cloud.gcs_storage_operations:JSON uploaded successfully


True

In [ ]:
//new changes from here 

In [14]:
encoding_metadata['encoders']['state']['value_to_encoded_mapping']

{'ak': np.int64(0),
 'al': np.int64(1),
 'ar': np.int64(2),
 'az': np.int64(3),
 'ca': np.int64(4),
 'co': np.int64(5),
 'ct': np.int64(6),
 'dc': np.int64(7),
 'de': np.int64(8),
 'fl': np.int64(9),
 'ga': np.int64(10),
 'hi': np.int64(11),
 'ia': np.int64(12),
 'id': np.int64(13),
 'il': np.int64(14),
 'in': np.int64(15),
 'ks': np.int64(16),
 'ky': np.int64(17),
 'la': np.int64(18),
 'ma': np.int64(19),
 'md': np.int64(20),
 'me': np.int64(21),
 'mi': np.int64(22),
 'mn': np.int64(23),
 'mo': np.int64(24),
 'ms': np.int64(25),
 'mt': np.int64(26),
 'nc': np.int64(27),
 'nd': np.int64(28),
 'ne': np.int64(29),
 'nh': np.int64(30),
 'nj': np.int64(31),
 'nm': np.int64(32),
 'nv': np.int64(33),
 'ny': np.int64(34),
 'oh': np.int64(35),
 'ok': np.int64(36),
 'or': np.int64(37),
 'pa': np.int64(38),
 'ri': np.int64(39),
 'sc': np.int64(40),
 'sd': np.int64(41),
 'tn': np.int64(42),
 'tx': np.int64(43),
 'ut': np.int64(44),
 'va': np.int64(45),
 'vt': np.int64(46),
 'wa': np.int64(47),
 '

In [15]:
/

SyntaxError: invalid syntax (1303184558.py, line 1)

In [ ]:
df_version_2.describe()

In [ ]:
# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
) 

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples\n")

In [19]:
# 2. Then scale
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  # Learn parameters from training data only
X_test_scaled = scaler.transform(X_test)        # Apply same parameters to test data

In [20]:
from sklearn.neighbors import KNeighborsRegressor



In [ ]:
print("Training Linear Regression model...")
lr_model =  KNeighborsRegressor(n_neighbors=5)
lr_model.fit(X_train_scaled, y_train)

In [22]:
# Make predictions
y_train_pred = lr_model.predict(X_train_scaled)
y_test_pred = lr_model.predict(X_test_scaled)

In [23]:
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
train_mae = mean_absolute_error(y_train, y_train_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

In [ ]:
print("\n=== MODEL PERFORMANCE ===")
print(f"Training RMSE: ${train_rmse:,.2f}")
print(f"Test RMSE: ${test_rmse:,.2f}")
print(f"Training MAE: ${train_mae:,.2f}")
print(f"Test MAE: ${test_mae:,.2f}")
print(f"Training R²: {train_r2:.4f}")
print(f"Test R²: {test_r2:.4f}")

In [ ]:
y_train.mean()

In [26]:
# from seaborn import histplot


# histplot(df['price'])

In [ ]:
df['type'].value_counts()

In [ ]:
df.groupby('type')['price'].mean()

In [ ]:
from seaborn import histplot


histplot(df['price'])